In [22]:
import os
import re
import pandas as pd
import shutil
import json
import difflib
import pprint as pp
from pathlib import Path
from io import StringIO
from itertools import zip_longest
from collections import defaultdict


### Funciones utiles para manejar archivos y directorios

In [23]:
def raiz():
    return Path.cwd().parent

def ubicar(comision, *camino):
    ''' Ubica una carpeta en la comisión indicada '''
    carpeta = f'lab4-C{comision}' if comision else 'lab4'  # Si la comisión es 0, se asume que es la cátedra
    comision_path = raiz() / carpeta
    return comision_path.joinpath(*camino)

def carpeta_alumno(alumno):
    ''' Genera el nombre de la carpeta del alumno '''
    return f"{alumno['orden']:02} - {alumno['legajo']} - {alumno['apellido']}, {alumno['nombre']}".strip()

def listar_carpetas(origen):
    return sorted([c for c in Path(origen).iterdir() if c.is_dir() and not c.name.startswith('.')])

def ubicar_alumno(legajo, *camino, alumnos=None):
    ''' Dado el legajo de un alumno, devuelve la carpeta donde debería estar '''
    if alumnos is None:
        alumnos = cargar_alumnos()

    alumno = alumnos.loc[alumnos['legajo'] == str(legajo)].iloc[0]
    comision = int(alumno['comision'])
    return ubicar(comision, 'practicos', carpeta_alumno(alumno), *camino)

def copiar_archivos(origen, destino, forzar=False):
    ''' Copia los archivos de una carpeta a otra '''
    origen_path = Path(origen)
    destino_path = Path(destino)

    # Si la carpeta de destino ya existe y se debe forzar la copia, eliminarla primero
    if destino_path.exists() and forzar:
        shutil.rmtree(destino_path)

    # Copiar todo el contenido de la carpeta de origen a la carpeta de destino
    shutil.copytree(origen_path, destino_path, dirs_exist_ok=True)

def renombrar_carpeta(origen, destino):
    ''' Renombra una carpeta '''
    origen_path = Path(origen)
    destino_path = Path(destino)

    # Renombrar la carpeta
    origen_path.rename(destino_path)

### Cargar alumnos

Toma los alumnos que estan en alumnos.md y lo carga en un datatable.

El formato del archivo debe ser
```
## Comision N
1. Legajo Apellido, Nombre
```

In [ ]:

def cargar_alumnos(origen='alumnos.md'):
    ''' Carga los alumnos desde el archivo de texto '''
    
    origen_path = Path(origen)
    
    with origen_path.open('r') as file:
        lineas = file.readlines()

    # Inicializar listas para almacenar los datos
    comisiones, ordenes, legajos, apellidos, nombres = [], [], [], [], []
    comision = None
    
    for linea in lineas:
        linea = re.sub(r'\|.*|^\s*|\s*$', '', linea)  # Limpiar la línea
        linea = linea[:60]
        if linea.startswith('## Comisión'):  # Extraer el número de la comisión
            comision = int(linea.split(' ')[-1])
        elif linea and linea[0].isdigit():  # Extraer el número de orden, legajo, apellido y nombre
            match = re.match(r'(\d+)\.\s+(\d+)\s+([^,]+),\s+(.+)', linea)
            if match:
                comisiones.append(comision)
                ordenes.append(int(match.group(1)))
                legajos.append(match.group(2))
                apellidos.append(match.group(3))
                nombres.append(match.group(4))
    
    # Crear el DataFrame
    data = {
        'comision': comisiones,
        'orden':    ordenes,
        'legajo':   legajos,
        'apellido': apellidos,
        'nombre':   nombres
    }
    return pd.DataFrame(data)

def guardar_alumnos(alumnos, destino='alumnos.md', asistencias=None):
    salida = StringIO()
    salida.write('# Alumnos Lab4.2024\n')

    for comision in alumnos['comision'].unique():
        salida.write("\n")
        salida.write(f'## Comisión {comision}\n')
        for _, alumno in alumnos[alumnos['comision'] == comision].iterrows():
            asistio = marcas(asistencias, alumno['legajo'])
            salida.write(f"{alumno['orden']:2}. {alumno['legajo']:5}  {alumno['apellido']}, {alumno['nombre']:40} {asistio}\n")

    destino_path = Path(destino)
    with destino_path.open('w') as file:
        file.write(salida.getvalue())

def marcas(asistencias, legajo):
    asistio = ''
    if asistencias and legajo in asistencias:
        for _, resultado in asistencias[legajo].items():
            if resultado['ausente']:
                asistio += '🔴'
            elif resultado['revisar']:
                asistio += '🔵'
            else:
                asistio += '🟢'
    return asistio

# print(cargar_alumnos())
cargar_alumnos()

### Detectar datos duplicados

Revisa si los legajos estan duplicados (error), y como curiosidad si hay nombre y apellidos repetidos.

In [4]:
def detectar_duplicados(data):
    def duplicados(columna):
        return data[data[columna].duplicated(keep=False)].sort_values(by=columna)
    
    def reportar_duplicados(columna):
        duplicados_columna = duplicados(columna)
        num_duplicados = len(duplicados_columna[columna].unique())
        if num_duplicados:
            print(f"\nHay {num_duplicados} {columna.title()} duplicados en {len(duplicados_columna)} registros")
            print(duplicados_columna)
        else:
            print(f"\nNo hay {columna.title()} duplicados")
    
    # Reportar registros con legajo duplicado
    print("Registros con legajo duplicado:")
    reportar_duplicados('legajo')
    
    # Reportar registros con nombre duplicado
    reportar_duplicados('nombre')
    
    # Reportar registros con apellido duplicado
    reportar_duplicados('apellido')

detectar_duplicados(cargar_alumnos())

Registros con legajo duplicado:

No hay Legajo duplicados

No hay Nombre duplicados

Hay 13 Apellido duplicados en 29 registros
     comision  orden legajo     apellido  \
4           2      5  58731      Alvarez   
5           2      6  58828      Alvarez   
6           2      7  59176    Argañaraz   
7           2      8  58909    Argañaraz   
38          5      2  58876         Díaz   
37          5      1  58690         Díaz   
47          5     11  58740       García   
46          5     10  59154       García   
53          5     17  59068     Gonzalez   
52          5     16  58720     Gonzalez   
21          2     22  55906     González   
20          2     21  55533     González   
51          5     15  59488     González   
19          2     20  58832     González   
56          5     20  58761       Juarez   
57          5     21  58758       Juarez   
62          5     26  58727      Lazarte   
61          5     25  58756      Lazarte   
102         7     12  59194       Me

### Generar trabajo prácticos
Copia de la carpeta prácticos el tp a cada una de los repositorios 


In [5]:
def publicar(practico):
    alumnos = cargar_alumnos()

    # Copia los archivos del práctico a cada alumno
    for legajo in alumnos['legajo']:
        destino = f"{ubicar_alumno(legajo, alumnos=alumnos)}/{practico}" 
        copiar_archivos(f'./practicos/{practico}', destino)
        
    # Copia el enunciados del práctico a cada comisión
    for comision in [2,5,7,9]:
        destino = ubicar(comision)
        shutil.copy(f'./practicos/enunciados/{practico}.md', f"{destino}/README.md")

# publicar('tp4')

### Normalizar nombres de carpetas.
Esta función permite revisar que los nombres estén bien escritos (tomando en cuenta el número de legajo).

In [6]:
def extraer_legajo(carpeta):
    ''' Extrae el legajo del nombre de la carpeta '''
    m = re.match(r'.*(\d{5}).*', str(carpeta))
    return m.group(1) if m else None

def normalizar():
    alumnos = cargar_alumnos()

    for comision in [2, 5 ,7, 9]:
        # listar todas las carpetas de alumnos que esten en la carpeta de la comisión

        carpetas = listar_carpetas(ubicar(comision, 'practicos'))
        for alumno in carpetas:
            legajo = extraer_legajo(str(alumno))
            if not legajo: continue
            
            origen  = ubicar(comision, 'practicos', alumno)
            destino = ubicar_alumno(legajo, alumnos=alumnos)
            
            if origen == destino: continue
            print(f"Renombrando [{origen}]\n            [{destino}]")

            renombrar_carpeta(origen, destino)
            
normalizar()


### Soluciona el problema que publique TP1 directamente en la raiz

In [7]:
def mover_ejercicio_a_tp1():
    alumnos = cargar_alumnos()
    for comision in [2, 5, 7, 9]:
        # Listar todas las carpetas de alumnos que estén en la carpeta de la comisión
        comision_path = ubicar(comision, 'practicos')
        carpetas = [carpeta for carpeta in Path(comision_path).iterdir() if carpeta.is_dir()]
        
        for carpeta in carpetas:
            carpeta_nombre = carpeta.name.strip()
            m = re.match(r'\d{2}\s+-?\s*(\d+) - ([^,]+), (.+)', carpeta_nombre)
            if not m:
                continue

            legajo = m.group(1)
            origen = ubicar_alumno(legajo, alumnos=alumnos)
            origen_path = Path(origen)

            archivos = [archivo for archivo in origen_path.iterdir() if archivo.is_file() and archivo.name != '.DS_Store']
            archivos.sort()

            tp1_path = origen_path / 'tp1'
            tp1_path.mkdir(exist_ok=True)

            for archivo in archivos:
                ruta_destino = tp1_path / archivo.name
                shutil.move(str(archivo), str(ruta_destino))
                print(f"Movido: {archivo} -> {ruta_destino}")

# mover_ejercicio_a_tp1()

### Funciones para comparar dos Jupiter Notebook 

In [21]:
def cargar_cuaderno(origen):
    '''Dado un archivo .ipynb, devuelve una lista con las líneas de código 
       de cada celda de código en el cuaderno.'''
    
    def limpiar(lineas): # Elimina líneas en blanco y comentarios
        return [linea.strip() for linea in lineas if linea.strip() and not linea.strip().startswith('#')]
    
    def codigo(cuaderno): # Extrae las líneas de código de las celdas de código
        celdas = cuaderno.get('cells', [])
        return [limpiar(celda.get('source', [])) for celda in celdas if celda.get('cell_type') == 'code']
    
    origen_path = Path(origen)
    with origen_path.open('r', encoding='utf-8') as f:
        cuaderno = json.load(f)

    return codigo(cuaderno)



def comparar_cuadernos(original, modificado):
    '''Dados dos cuadernos de Jupyter, devuelve la cantidad 
       de líneas de código que se agregaron en el segundo 
       respecto del primero.'''

    def igualar(a, b): 
        while len(a) < len(b): 
            a.append([])
        while len(b) < len(a): 
            b.append([])
        return zip(a, b)
    
    def cambios(a, b): 
        diferencias = difflib.unified_diff(a, b, lineterm='')
        diferencias = [l for l in diferencias if l.startswith('+') and not l.startswith('+++')]
        return len(diferencias)
    
    conteo = [cambios(o, m) for o, m in igualar(original, modificado)]
    
    # def cambios(origen, modificado):
    #     diferencias = difflib.unified_diff(origen, modificado, lineterm='')
    #     return sum(1 for l in diferencias if re.match(r'^\+[^+]', l))
    
    # conteo = [cambios(o, m) for o, m in zip_longest(original, modificado, fillvalue=[])]

    return {'diferencias': conteo, 'total': sum(conteo)}


In [9]:
def listar_cuadernos(origen):
    '''Dada una carpeta, devuelve una lista con los cuadernos de Jupyter (.ipynb) que contiene.'''
    return [archivo for archivo in Path(origen).iterdir() if archivo.suffix == '.ipynb']

def analizar(practico, comisiones=[2, 5, 7, 9]):
    if not isinstance(comisiones, list):
        comisiones = [comisiones]

    cuadernos = listar_cuadernos(ubicar(0, 'practicos', practico))
    print(f"Analizando {practico}...")
    print(f'{cuadernos!s}')

# analizar('tp1')
# analizar('tp2')
analizar('tp3')


Analizando tp3...
[PosixPath('/Users/adibattista/Documents/GitHub/lab4/practicos/tp3/ejercicio.ipynb')]


In [20]:
def analizar_cambios(practicos, comisiones=[2, 5, 7, 9]):
    resultados = defaultdict(dict)
    if not isinstance(comisiones, list): comisiones = [comisiones]
    if not isinstance(practicos, list):  practicos  = [practicos]

    normalizar()

    for practico in practicos:

        archivos = listar_cuadernos(ubicar(0, 'practicos', practico))
        for archivo in archivos:
            ruta_original = ubicar(0, 'practicos', practico, archivo)

            print(f"Analizando {ruta_original}:")
            original = cargar_cuaderno(ruta_original)
            celdas   = len(original)

            for comision in comisiones:
                carpetas = listar_carpetas(ubicar(comision, 'practicos'))
                print(f"Comisión {comision}: {len(carpetas)} alumnos")

                for alumno in carpetas:
                    print(f'  {alumno.name:50} [{alumno}]')
                    destino = ubicar(comision, 'practicos', alumno, practico, archivo)
                    cambios = comparar_cuadernos(original, cargar_cuaderno(destino))

                    legajo = extraer_legajo(alumno)
                    cambios['celdas']  = celdas
                    cambios['ausente'] = cambios['total'] < 5
                    cambios['revisar'] = max(cambios['diferencias']) >= 40
                    resultados[legajo][practico] = cambios

    return resultados

alumnos = cargar_alumnos()
# r = analizar_cambios(['tp1','tp2','tp3'])
r = analizar_cambios('tp2', [2, 5, 7, 9])
pp.pprint(r)

# guardar_alumnos(alumnos, asistencias=r)


Analizando /Users/adibattista/Documents/GitHub/lab4/practicos/tp2/Ejercicio.ipynb:
Comisión 2: 37 alumnos
  01 - 58764 - Acuña, Ana Sofía                      [/Users/adibattista/Documents/GitHub/lab4-C2/practicos/01 - 58764 - Acuña, Ana Sofía]
  02 - 59277 - Aguirre, Emanuel                      [/Users/adibattista/Documents/GitHub/lab4-C2/practicos/02 - 59277 - Aguirre, Emanuel]
  03 - 58827 - Albornoz Silva, Alejo Miguel          [/Users/adibattista/Documents/GitHub/lab4-C2/practicos/03 - 58827 - Albornoz Silva, Alejo Miguel]
  04 - 58952 - Almiron, Maicol Leonel                [/Users/adibattista/Documents/GitHub/lab4-C2/practicos/04 - 58952 - Almiron, Maicol Leonel]
  05 - 58731 - Alvarez, Nicolás                      [/Users/adibattista/Documents/GitHub/lab4-C2/practicos/05 - 58731 - Alvarez, Nicolás]
  06 - 58828 - Alvarez, Nicolás Nahuel               [/Users/adibattista/Documents/GitHub/lab4-C2/practicos/06 - 58828 - Alvarez, Nicolás Nahuel]
  07 - 59176 - Argañaraz, Facundo N